In [1]:
import pandas as pd
import numpy
import csv
import re


## train(20170607~20200306 ) 최종 자산이 제일 높을 때를 구함.

stock =["유한양행","씨젠","오스템임플란트","차바이오텍","헬릭스미스","메지온","에이치엘비생명과학","대웅","녹십자홀딩스",
"동국제약","CMG제약","영진약품","메디포스트","코미팜","엔케이맥스","바디텍메드","JW중외제약","클래시스","종근당홀딩스",
"서흥","카이노스메드","앱클론","아이센스","에이프로젠제약","바텍","삼진제약","동화약품","수젠텍","제일약품",
"뷰웍스","테라젠이텍스","한독","케어젠","대원제약","유비케어","일동제약","동성제약","지노믹트리","제일파마홀딩스","휴마시스",
        "환인제약","진원생명과학","종근당바이오","종근당","제넥신","일양약품","이연제약","유나이티드제약","오리엔트바이오"
        ,"셀트리온제약","삼천당제약","삼성제약","삼성바이오로직스","부광약품","명문제약","대웅제약","국제약품","광동제약"
        ,"고려제약","경보제약",
     "메디톡스","아미코젠","인트론바이오","알테오젠","한미사이언스","바이오니아","레고켐바이오","셀트리온","셀트리온헬스케어","안트로젠",
"유바이오로직스","보령제약","파마리서치","엔지켐생명과학","KPX생명과학","에스티팜","휴온스","한미약품","한올바이오파마","지트리비앤티",
        "녹십자랩셀","올릭스",
"크리스탈지노믹스","휴젤","셀리버리","이수앱지스","바이넥스","휴메딕스","휴온스글로벌"]




news_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\title_ratio_all/title_ratio_pororo_after_오스템임플란트.txt', delimiter = '\t')
discuss_P_N = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\제약테마_감성분석완료\discuss_ratio_all/discuss_title_ratio_pororo_after2_오스템임플란트.txt', delimiter = '\t')



stock_data2 = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\주식데이터/오스템임플란트_일봉_train.csv',encoding='cp949')

stock_data2=stock_data2.drop(columns=['기업명','기업코드','시간','시가','고가','저가','거래량'])
stock_data2.rename(columns={'날짜':'id'},inplace=True)



max_account=0;
fit_title_num=0;
fit_ratio=0;
fit_discuss_positive=0;

for title_num in range(30):
    for ratio in range(80):
        for discuss_positive in range(20):
            long_id_news = news_P_N[news_P_N['num']>title_num][news_P_N['ratio']>ratio][news_P_N['id']<"20200306"].id

            long_id_news = long_id_news.reset_index(drop=True)

            for i in range(len(long_id_news)):
                long_id_news[i] = re.sub("\.","",long_id_news[i])

            long_id_discuss = discuss_P_N[discuss_P_N['positive']>discuss_positive].id

            long_id_discuss = long_id_discuss.reset_index(drop=True)

            for i in range(len(long_id_discuss)):
                long_id_discuss[i] = re.sub("\.","",long_id_discuss[i])


            real_long_id=[]

            # 위에 해당하는 news의 조건과 종목토론방 조건이 맞는 날에 대해서 뽑음
            for i in long_id_news:
                for k in long_id_discuss:
                    if i==k:
            #             print(i)
                        real_long_id.append(i)

            right = 0
            right_ratio=[]
            false = 0
            false_ratio=[]

            account = 10000000
            left = 0

            buy = 0
            sell = 0 
            num = 0


            for i in real_long_id:
                for k in range(len(stock_data2)):
                    if(i == str(stock_data2['id'][k]) and stock_data2['id'][k]!=20200306):
                        # 만약에 long을 맞춘 경우
                        if(stock_data2['등락률'][k] >= 0):
                            right+=1
                            right_ratio.append(stock_data2['등락률'][k])
                        # long으로 예상했는데 틀린경우
                        else:
                            false+=1
                            false_ratio.append(stock_data2['등락률'][k])

#                         print("실제 다음날 등락률 : ",stock_data2['등락률'][k])
                        buy = stock_data2['종가'][k]*(account//(stock_data2['종가'][k]*1.00015))
                        num = (account//(stock_data2['종가'][k]*1.00015))
#                         print("구매 수:",num)
                        # 다음날이 -1 임!! 
                        sell = stock_data2['종가'][k-1]*num
#                         print("구매가격 : ",stock_data2['종가'][k])
#                         print("구매가치 : ",buy)
                        left = account - buy
#                         print("잔고--   : ",left)
#                         print("판매가격 : ",stock_data2['종가'][k-1])
#                         print("판매가치 : ",sell)

                        account = sell * 0.9935 + left
#                         print("현재 자산 : ",account)

#             print(right_ratio)
#             print(false_ratio)
            print("기사수 : ", title_num)
            print("기사긍정비율 : ",ratio)
            print("긍정토론수 : ",discuss_positive)
            
            if((right + false)!=0):
                print("acc = ",right/(right + false) * 100 ,"%")
                print("최종 account : ",account)
                
            print("현재 갯수 : ",(right + false))
            
            # 예측이 5개 이상인 경우만 판단
            if (account > max_account and (right + false) >= 5 ):
                max_account = account
                fit_title_num=title_num;
                fit_ratio=ratio;
                fit_discuss_positive=discuss_positive;
            
            print("현재 max : ",max_account)
                
                

                
# 가장 높게 나온 최종 값과 해당 임계값들 출력
print("가장 최적 account : ",max_account)
print("가장 최적 조건 : ",fit_title_num,fit_ratio,fit_discuss_positive)


                

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Desktop\\Desktop\\주가예측\\제약테마_감성분석완료\\title_ratio_all/title_ratio_pororo_after_오스템임플란트.txt'